In [2]:
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import seaborn as sns
import numpy as np

In [4]:
plates = 'RBP201202A147|RBP201202A134|RBP201202A135|RBP201202A136|RBP201202A054|RBP201202A053|RBP201202A052|RBP201202A051'

triplicate = pd.read_pickle('/ghds/groups/labdesk/bshih/c19dash/c19_dashboard/c19_call.pickle').query("runid == '210122_NB552482_0010_AHNY33BGXG'").dropna(axis=1, how='all')
triplicate = triplicate[triplicate.pos_tube_rack.str.contains(plates, na=False)]

In [5]:
fast = 'RBP201202A147|RBP201202A134|RBP201202A135|RBP201202A136'
RBP1 = 'RBP201202A147|RBP201202A135|RBP201202A054|RBP201202A052'
nasal = 'RBP201202A147|RBP201202A134|RBP201202A054|RBP201202A053'

map1 = {'A':0, 'B':0,
        'C':125, 'D':125,
        'E':125, 'F':12.5,
        'G':12.5, 'H':12.5}

map2 = {'A':0, 'B':0,
        'C':50, 'D':50,
        'E':50, 'F':25,
        'G':25, 'H':25}

In [7]:
triplicate

,runid,run_sample_id,sample_type,pos_tube_rack,parameter_set,replicates_count,replicates_detected,replicates_not_detected,replicates_no_call,median_covid_ratio,call,flags,extraction,background,copies
96,210122_NB552482_0010_AHNY33BGXG,4052431982,Sample,RBP201202A051:A11,Guardant19-RTPCR,3.0,0.0,3.0,0.0,0.00000,not_detected,pass,norm,saliva,0.0
97,210122_NB552482_0010_AHNY33BGXG,4052432027,Sample,RBP201202A051:E8,Guardant19-RTPCR,3.0,3.0,0.0,0.0,0.18106,detected,pass,norm,saliva,50.0
98,210122_NB552482_0010_AHNY33BGXG,4052432030,Sample,RBP201202A051:E11,Guardant19-RTPCR,3.0,3.0,0.0,0.0,0.13113,detected,pass,norm,saliva,50.0
99,210122_NB552482_0010_AHNY33BGXG,4052432008,Sample,RBP201202A051:D1,Guardant19-RTPCR,3.0,3.0,0.0,0.0,0.21902,detected,pass,norm,saliva,50.0
100,210122_NB552482_0010_AHNY33BGXG,4052432021,Sample,RBP201202A051:E2,Guardant19-RTPCR,3.0,3.0,0.0,0.0,0.40378,detected,pass,norm,saliva,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,210122_NB552482_0010_AHNY33BGXG,4052459703,Sample,RBP201202A054:G12,Guardant19-RTPCR,3.0,2.0,1.0,0.0,0.03046,detected,pass,norm,nasal,12.5
860,210122_NB552482_0010_AHNY33BGXG,4052459714,Sample,RBP201202A054:H11,Guardant19-RTPCR,3.0,2.0,1.0,0.0,0.02454,detected,pass,norm,nasal,12.5
861,210122_NB552482_0010_AHNY33BGXG,4052459715,Sample,RBP201202A054:H12,Guardant19-RTPCR,3.0,3.0,0.0,0.0,0.03206,detected,pass,norm,nasal,12.5
862,210122_NB552482_0010_AHNY33BGXG,N2008260173,NTC,RBP201202A054:A1,Guardant19-RTPCR,3.0,0.0,3.0,0.0,0.00000,not_detected,pass,norm,nasal,0.0


In [6]:
triplicate.loc[triplicate.pos_tube_rack.str.contains(fast), 'extraction'] = 'fast'
triplicate.loc[~triplicate.pos_tube_rack.str.contains(fast), 'extraction'] = 'norm'

triplicate.loc[triplicate.pos_tube_rack.str.contains(nasal), 'background'] = 'nasal'
triplicate.loc[~triplicate.pos_tube_rack.str.contains(nasal), 'background'] = 'saliva'

triplicate.loc[triplicate.pos_tube_rack.str.contains(RBP1), 'copies'] = triplicate.pos_tube_rack.map(lambda x: x.split(':')[1][0]).map(map1)
triplicate.loc[~triplicate.pos_tube_rack.str.contains(RBP1), 'copies'] = triplicate.pos_tube_rack.map(lambda x: x.split(':')[1][0]).map(map2)

In [8]:
pos_ctrl = triplicate[triplicate['run_sample_id'].str.contains(r'^POS.*')]
no_ctrl = triplicate[triplicate['run_sample_id'].str.contains(r'^\d{10}')]
no_ctrl.loc[:, 'time'] = 'new'

y = pd.get_dummies(no_ctrl.call)

a = pd.concat([no_ctrl, y], axis=1).groupby(['extraction', 'background', 'copies']).agg({'detected':'sum',
                                                                                     'not_detected':'sum'})
a.loc[:,'total'] = a['detected'] + a['not_detected']
a.loc[:,'percent_detect'] = ((a['detected']/a['total'])*100).round(2)
a

/home/bshih/miniconda3/envs/c19dash/lib/python3.6/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/bshih/miniconda3/envs/c19dash/lib/python3.6/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


detected  not_detected  total  percent_detect
extraction background copies                                               
fast       nasal      0.0            1            43     44            2.27
                      12.5          12            24     36           33.33
                      25.0          26            10     36           72.22
                      50.0          34             2     36           94.44
                      125.0         36             0     36          100.00
           saliva     0.0            0            44     44            0.00
                      12.5          26            10     36           72.22
                      25.0          34             2     36           94.44
                      50.0          36             0     36          100.00
                      125.0         36             0     36          100.00
norm       nasal      0.0            0            44     44            0.00
                      12.5          23            13     36           63.89
                      25.0          36             0     36          100.00
                      50.0          36             0     36          100.00
                      125.0         36             0     36          100.00
           saliva     0.0            1            43     44            2.27
                      12.5          28             8     36           77.78
                      25.0          34             2     36           94.44
                      50.0          36             0     36          100.00
                      125.0         36             0     36          100.00

In [6]:
no_ctrl

,runid,run_sample_id,sample_type,pos_tube_rack,parameter_set,replicates_count,replicates_detected,replicates_not_detected,replicates_no_call,median_covid_ratio,call,flags,extraction,background,copies,time
96,210122_NB552482_0010_AHNY33BGXG,4052431982,Sample,RBP201202A051:A11,Guardant19-RTPCR,3.0,0.0,3.0,0.0,0.00000,not_detected,pass,norm,saliva,0.0,new
97,210122_NB552482_0010_AHNY33BGXG,4052432027,Sample,RBP201202A051:E8,Guardant19-RTPCR,3.0,3.0,0.0,0.0,0.18106,detected,pass,norm,saliva,50.0,new
98,210122_NB552482_0010_AHNY33BGXG,4052432030,Sample,RBP201202A051:E11,Guardant19-RTPCR,3.0,3.0,0.0,0.0,0.13113,detected,pass,norm,saliva,50.0,new
99,210122_NB552482_0010_AHNY33BGXG,4052432008,Sample,RBP201202A051:D1,Guardant19-RTPCR,3.0,3.0,0.0,0.0,0.21902,detected,pass,norm,saliva,50.0,new
100,210122_NB552482_0010_AHNY33BGXG,4052432021,Sample,RBP201202A051:E2,Guardant19-RTPCR,3.0,3.0,0.0,0.0,0.40378,detected,pass,norm,saliva,50.0,new
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,210122_NB552482_0010_AHNY33BGXG,4052459679,Sample,RBP201202A054:E12,Guardant19-RTPCR,3.0,3.0,0.0,0.0,0.18310,detected,pass,norm,nasal,125.0,new
858,210122_NB552482_0010_AHNY33BGXG,4052459691,Sample,RBP201202A054:F12,Guardant19-RTPCR,3.0,2.0,1.0,0.0,0.03089,detected,pass,norm,nasal,12.5,new
859,210122_NB552482_0010_AHNY33BGXG,4052459703,Sample,RBP201202A054:G12,Guardant19-RTPCR,3.0,2.0,1.0,0.0,0.03046,detected,pass,norm,nasal,12.5,new
860,210122_NB552482_0010_AHNY33BGXG,4052459714,Sample,RBP201202A054:H11,Guardant19-RTPCR,3.0,2.0,1.0,0.0,0.02454,detected,pass,norm,nasal,12.5,new


In [9]:
brush = alt.selection(type = 'interval')

points = alt.Chart(no_ctrl).mark_point().encode(
    column = alt.Column('copies:N',  header = alt.Header(labelOrient = "bottom")),
    x = alt.X('extraction:N'),
    y = alt.Y('median_covid_ratio', scale = alt.Scale(type = 'log', base = 10, domain = [0.0001, 10])),
    tooltip=['extraction', 'background', 'replicates_detected', 'median_covid_ratio'],
    color= alt.Color('replicates_detected:O', scale=alt.Scale(scheme='dark2'))
).properties(
    width = 75,
    height = 300
).facet(
    facet='background:N',
    columns=2
).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).interactive()

points 

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [ ]:
g = sns.catplot(y="median_covid_ratio", x="copies", col="background",
                kind="strip", orient="v", height=5, aspect=0.9,
                data=no_ctrl.query("median_covid_ratio != 0"), hue='extraction', 
                color='#b19cd9', dodge=True, jitter=True)

g.tight_layout()
g.set(yscale="log")
[i.axhline(linewidth=1.5, color='r', ls='--', y=0.01) for i in g.axes[0]]
plt.show()

In [ ]:
g = sns.catplot(y="median_covid_ratio", x="copies", col="background",
                kind="box", orient="v", height=5, aspect=0.9,
                data=no_ctrl.query("median_covid_ratio != 0"), hue='extraction', 
                color='#b19cd9', linewidth=2.5)
g.tight_layout()
g.set(yscale="log")
[i.axhline(linewidth=1.5, color='r', ls='--', y=0.01) for i in g.axes[0]]
plt.show()

In [ ]:
mapping = {'A':0, 'B':0,
           'C':100, 'D':100,
           'E':50, 'F':50,
           'G':25, 'H':25
            }

df = pd.read_pickle('./c19dash/c19_dashboard/c19_call.pickle').dropna(subset=['pos_tube_rack']).query("sample_type == 'Sample'").reset_index(drop=True)
df.drop(df[df['pos_tube_rack'].str.contains(r'(RBP201022A059|RBP201022A057|RBP201022A176|RBP201022A179):.(6|7|8|9|10|11|12)')].index, inplace=True)

r = re.compile(r"(RBP201022A060|RBP201022A177|RBP201022A178|RBP201022A088|RBP201022A090|RBP201022A109|RBP201022A110|RBP201022A189|RBP201022A190)")
df_nasal = df[df['pos_tube_rack'].str.contains(r)].dropna(how='all', axis=1)
df_nasal.loc[:, 'background'] = 'nasal'
df_nasal.loc[:, 'copies'] = df_nasal.pos_tube_rack.map(lambda x: x.split(':')[1][0]).map(mapping)

r = re.compile(r"(RBP201022A057|RBP201022A176|RBP201022A086|RBP201112A131|RBP201022A108|RBP201022A185|RBP201022A188|RBP201022A241)")
df_spittoon = df[df['pos_tube_rack'].str.contains(r)].dropna(how='all', axis=1)
df_spittoon.loc[:, 'background'] = 'saliva'
df_spittoon.loc[:, 'copies'] = df_spittoon.pos_tube_rack.map(lambda x: x.split(':')[1][0]).map(mapping)

In [ ]:
total = pd.concat([df_nasal, df_spittoon], axis=0)
total.loc[:, 'extraction'] = 'fast'
total.loc[:, 'time'] = 'old'

In [ ]:
everything = pd.concat([no_ctrl, total], axis=0)

g = sns.catplot(y="median_covid_ratio", x="copies", col="background",
                kind="box", orient="v", hue='time', height=5, aspect=0.9,
                data=everything.query("median_covid_ratio != 0"), 
                showfliers=True, color='#b19cd9', linewidth=2.5)

g.tight_layout()
g.set(yscale="log")
[i.axhline(linewidth=1.5, color='r', ls='--', y=0.01) for i in g.axes[0]]
plt.show()